In [1]:
import requests
from bs4 import BeautifulSoup
import json 
import pymysql
import time

In [2]:
def mtUrl(search):
    url_temp = 'https://search.mt.co.kr/searchNewsList.html?srchFd=TOTAL&range=IN&reSrchFlag=&preKwd=&search_type=m&kwd={search}&bgndt=20190401&enddt=20190930&category=MTNW&sortType=allwordsyn&subYear=&category=MTNW&subType=mt'.format(search=search)
    for pageNo in range(1, 31):
        url = url_temp + "&pageNum={pageNo}".format(pageNo=pageNo)
        html = requests.get(url)  
        bs = BeautifulSoup(html.content,"html.parser")
        time.sleep(2)
        news_urls = []
        for url_no in range(len(bs.select(".section .con .subject"))):
            news_urls.append(bs.select(".section .con .subject")[url_no].select("a")[0].get("href"))
        mtInfo(news_urls)

In [7]:
def mtInfo(news_urls):
    for url_no in range(len(news_urls)):
        url2 = news_urls[url_no]
        html2 = requests.get(url2)
        bs2 = BeautifulSoup(html2.content,"html.parser")
        time.sleep(2)
        try:  
            # 제목
            title = bs2.select(".subject")[0].text
            
            # 부제목
            if bs2.select(".sub_subject") == []:
                sub_title=""
            else:
                sub_title = bs2.select(".sub_subject")[0].text
            
            # 날짜
            newsDate = bs2.select(".date")[0].text
            newsDate = newsDate.replace('"', '').replace(".", "")[0:8]
            print(newsDate)
            
            # 본문
            content = []
            for content_no in range(len(bs2.select("#gisa_section .view_text #textBody"))):
                content.append(bs2.select("#gisa_section .view_text #textBody")[content_no].text)
            content_replace = content[0].replace("\n","").replace("\t","").replace("\r","")
            
            print(url2)
            print("제목 : ",title)
            print("부제목 : ",sub_title)
            print(newsDate)
            print("본문 :",content_replace)
            print("============================================================")
        except:
            print("<<<<<에러 url:>>>>>>", url2)
            continue
        
        dbData = [[url2, title, newsDate, sub_title, content_replace]]
        connectDB(dbData)

In [4]:
def connectDB(dbData):
    DB_HOST = '127.0.0.1'
    DB_USER = 'root'
    DB_PASSWD = 'autoset'
    DB_NAME = 'final'
    
    conn = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWD,
                          db=DB_NAME, charset='utf8')
    curs = conn.cursor()
    sql = """insert into mt_bs(url, title, date, sub_title, contents) values (%s, %s, %s, %s, %s)"""
    curs.executemany(sql,dbData)
    conn.commit()
    
    conn.close()

In [5]:
search = '%C0%CF%BA%BB+%BF%A9%C7%E0'
# 보통 우리가 무엇을 검색해서 크롤링 할 지 정해져있기 때문에 미리 저장해둠.
mtUrl(search)

https://news.mt.co.kr/mtview.php?no=2019093016232219020&type=1
제목 :  일본 대신 국내로…'호캉스' 떠난 사람들
부제목 :  문화체육관광부, 한국호텔업협회 8월 주요 지역 관광호텔 조사…객실이용률 79.9%, 전년 比 3.4%↑
20190930
본문 : 호캉스. /사진=인터콘티넨탈 서울 파르나스여름 휴가철 '일본여행 보이콧'의 반사이익이 국내 호텔로 향했다. 불매운동 효과가 본격화한 8월 한 달 동안, 여행객들이 '호캉스(호텔+바캉스)'를 노리고 호텔로 발길을 돌리면서 국내 관광호텔의 객실이용률이 상승한 것으로 나타났다.30일 문화체육관광부가 서울과 제주, 부산 등 국내 주요지역에 위치한 1~5성급 관광호텔 200곳을 대상으로 8월 운영 현황을 조사한 결과, 전년 동월 대비 객실이용률과 객실판매단가, 외국인 판매객실비율이 모두 상승했다고 밝혔다.조사 결과에 따르면 전체 평균 객실이용률은 79.9%로 지난해보다 3.4%, 2017년 8월보다 8% 상승했다. 특히 5성급 특급호텔의 객실이용률이 84.8%로 모든 성급에서 가장 높게 나타난 점이 눈에 띈다. 지난해보다 5.8% 높아진 수치다.이는 호캉스(호텔+바캉스) 트렌드가 보편적인 여가문화로 자리잡았기 때문이라는 설명이다. 비용부담은 있지만 이동시간이 짧고 서비스 수준이 높은 데다, 수영장 등 다양한 부대시설이 마련돼 있어 선호도가 높아졌다는 것이다.5성급에 이어 고급호텔로 분류되는 4성급 호텔도 83%로 높은 이용률을 기록했고, 3성급 호텔도 78.9%로 80%에 육박했다. 두 성급 모두 지난해보다 객실 이용률이 각각 3.6%, 1.3% 상승했다.이용률 증가로 객실 판매단가도 오름세를 보였다. 평균 객실 판매단가는 13만1251원으로 지난해와 비교해 4.1% 상승했다. 최근 수년 새 호텔공급 증가로 객실 판매단가가 떨어져 운영에 어려움을 겪는 호텔이 늘고 있다는 지적을 감안하면 긍정적인 결과다. 객실 총매출액도 996억 원으로 909억 원을 기록한 지난해보다 

https://news.mt.co.kr/mtview.php?no=2019092914132931954&type=1
제목 :  日서 열린 '韓문화축제' 북적… "교류는 계속돼야"
부제목 :  28~29일 한일축제한마당에 8.2만명 다녀가 日 관광 담당 장관 "교류는 계속돼야" 강조
20190929
본문 : 28~29일 일본 도쿄에서 열린 '한일축제한마당(일본명 일한교류축제)'에서 한국 연무단이 대태권도를 활용한 춤을 선보이고 있다. 이번 행사에는 약 8만2000명의 관람객이 다녀간 것으로 추산된다. /사진=NHK 갈무리일본의 수출규제로 한국과 일본 관계가 최악의 상황으로 치닫고 있는 가운데 일본에서 열린 양국 문화교류축제에 수만 명의 관람객이 몰렸다. 일본 NHK방송에 따르면 28~29일 일본 도쿄도 지요다구 히비야공원에서 열린 '한일축제한마당(일본명 일한교류축제)' 행사에 약 8만2000명이 참여했다. 역대 가장 많은 관람객이 찾았던 지난해와 비슷한 수준이다. NHK는 "한국과 일본의 정치적 갈등이 심해지는 가운데 행사에 대한 우려도 있었지만, 행사 규모나 관람객은 예년과 비슷했다"고 전했다. 이 행사는 2005년 한일 수교 40주년을 기념해 처음 열렸으며, 2009년 이후 매년 이맘때쯤 서울과 도쿄에서 열리는 것으로 정례화됐다. 전날 개회식에는 아키바 가즈요시 일본 국토교통상, 나카야마 노리히로 일본 외무성 정무관이 참석했다. 우리나라에서는 노태강 한국 문화체육관광부 2차관과 남관표 주일본 한국대사 등이 함께했다. 관광정책 담당 장관인 아키바 국토교통상은 이날 한일 갈등으로 일본을 찾는 한국인 여행객이 크게 줄었다는 점을 언급하며 "양국 관계의 미래를 위해 국민 간 교류나 문화교류는 계속돼야 한다"면서 "민간 교류가 활발하다면 양국 우호관계가 조금도 흔들리지 않을 것"이라고 말했다. 남 대사는 "이번 행사를 통해 양국 다음 세대의 우정과 이해가 더욱 깊어지길 바란다"고 전했다. 행사에서는 우리나라 전직 태권도 국가대표 선수로 구성된 연무단이 태권도를 활용한 춤

https://news.mt.co.kr/mtview.php?no=2019092608471432856&type=1
제목 :  [광화문]두 개의 불매운동
부제목: 없음
20190927
본문 : "한국에서 벌어지고 있는 불매운동으로 일본 기업이 큰 피해를 입고 있다." (사사키 미키오 일한경제인회 회장) 25일 서울에서 폐막한 제51회 한일경제인회의에서 일본 재계 인사들은 불안한 마음을 숨기지 않았다. 양국에서 300여 명이 참석한 한일 경제전쟁 이후 첫 대규모 회동이었다. 한국과의 갈등에도 불구하고 별다른 피해가 없다는 게 일본 정부의 공식 입장. 하지만 회의 참석자들은 "지금까지 마주하지 못했던 매우 어려운 국면"(고가 노부유키 노무라홀딩스 회장), "수출 규제로 파급과 충격이 있지만 공개할 수 없을 뿐"(오쿠다 사토루 아시아대 교수)이라고 털어놓았다.일본 인사들은 특히 하나같이 한국에서 거세게 불고 있는 불매운동의 충격을 거론하며 "어려운 시기일수록 경제협력이 계속돼야 한다"고 호소했다. "한국은 금방 뜨거워지고 금방 식는 나라로 불매운동은 오래가지 않을 것"이라던 분쟁 초기의 호기롭던 반응을 찾아보기 어려웠다. 이 같은 반응은 8월에 일본을 찾은 한국인 여행자 수가 반토막(48%) 나는 등 불매운동 영향이 갈수록 커지고 있기 때문이다. 요미우리, 아사히, 산케이 등 일본 6대 일간지 중 4곳이 1면에 게재할 정도로 일본 사회에 큰 충격을 줬다. 급기야 일본 정부는 한국 방문객이 90% 이상 줄어 렌터카·숙박 폐업 업체가 속출하는 쓰시마섬에 긴급 재정을 투입하기로 했다. 일본 불매운동이 이처럼 큰 힘을 발휘한 것은 국민의 광범위한 공감대 속에 자발적인 참여가 있었기 때문이다. 평범한 시민들의 양보와 인내가 한국 정부의 항의에도 꿈쩍 않던 일본 정부와 재계에 영향을 미쳤다는 점에서 큰 의미가 있다.이번 성공은 최근 여론 지탄 속에 무산된 불매운동의 반면교사라는 점에서도 주목된다. 업계 초유로 자사 차량 불매운동에 나섰던 한국GM 노조가 그 주인공이다. 노조는 사

https://news.mt.co.kr/mtview.php?no=2019092415054757841&type=1
제목 :  '보이콧 재팬'에 항공업계 내년 실적도 '암울' 
부제목 :  한국기업평가 웹 세미나-비우호적 단기업황에 내년 실적 저하도 불가피
20190924
본문 : 일본 여행 거부 운동이 장기화하는 가운데 항공업계의 실적 감소가 내년까지 계속될 것이란 전망이 나왔다. 한일 노선 수요 부진이 계속되고 동남아시아 등 대체노선 효과가 충분치 않을 수 있다는 설명이다.지광훈 한국기업평가 기업본부 평가3실 수석연구원은 지난 8일 진행한 웹 세미나에서 '한일 갈등, 갈 곳 잃은 항공업계' 주제발표를 통해 "항공업계의 항공기 변경, 감편, 운항 중단 등의 대응방안의 실효성은 제한적"이라고 주장했다. 이에 한기평은 항공 업황에 대한 전망을 기존 '중립적'에서 '비우호적'으로 변경했다.지 연구원은 항공사들의 올해 실적 부진이 예상보다 클 수 있다고 밝혔다. 그는 △환율, 유가 등 비우호적 영업환경 △일본 사태로 인한 전통적 여객 성수기인 3분기 타격 △일본 여행수요가 많은 추석 명절효과 상실 △대체노선 효과의 점진적 반영 등을 이유로 들었다.일본 여행 거부 운동에 따른 매출 감소율은 FSC(대형항공사)보다 LCC(저비용항공사)가 더 컸다. FSC인 대한항공 (17,850원 250 1.4%)과 아시아나 (3,870원 30 0.8%)항공의 올해 매출 감소율은 전체 매출의 0.5~1.6% 내외(327억~1386억원)로 예상했다. 대체 노선 등이 있는 FSC는 상대적으로 제한적인 수준의 실적 저하가 나타날 것으로 분석했다.LCC에 대해선 예상보다 큰 폭의 실적 저하를 전망했다. LCC는 대부분이 단일 기종으로 구성되거나 주력 항공기 비중이 80% 이상을 차지해 기재 변경의 방안이 큰 의미를 갖지 못한다. 그는 "감편이나 운항 중단이 길어지면 기존 슬롯(시간당 항공기 이착륙 횟수)을 유지할 수 없다는 문제가 있다"며 "실적 변동성과 고효율 한일 노선에 기반을 둔 LCC들의

https://news.mt.co.kr/mtview.php?no=2019092313482570452&type=1
제목 :  한국인은 안가도 '한국 온 일본 관광객' 늘었다
부제목 :  8월 일본인 33만여명 한국 방문, 4.6% 증가
20190923
본문 : 지난 8일 오후 서울 남산을 찾은 외국인 관광객이 맑고 청명한 하늘과 서울시내를 바라보고 있다./사진=뉴스1일본의 수출규제로 촉발한 한일관계가 연일 악화일로를 걷고 있지만, 지난달 일본 관광객들의 한국행 발걸음은 오히려 늘어난 것으로 나타났다.23일 한국관광공사가 발표한 한국관광통계에 따르면 지난 8월 한국을 방문한 외국인은 총 158만6299명으로 전년 동월 대비 14% 증가했다.중국이 전반적인 방한 관광시장의 성장세를 견인했다. 가족여행, 학생단체 방문이 늘면서 총 57만8112명이 방문, 지난해 같은 기간보다 20.9% 증가했다.일본도 성장세를 이어갔다. 지난 7월 촉발한 일본 정부의 수출규제로 한일관계가 악화한 상황에서도 32만9652명이 한국을 찾아 4.6% 늘어났다. 하지만 한일갈등의 여파로 매월 두 자릿수를 구가하던 성장세가 한 자릿수로 꺾였다.이 밖에 아시아‧중동 지역의 관광객도 전년 동월 대비 12.9% 증가했다. 대만은 개별여행(FIT) 수요 증가 및 항공 노선 증편 등에 따라 10만9273명이 방문, 전년 대비 8.7% 성장했다. 홍콩 역시 정치적 불안정에도 불구하고 6만5958명이 방문해 4% 증가했다. 마카오는 6321명으로 1.5% 소폭 하락했다.최근 방한 관광시장에서 주목받는 동남아시장도 견고한 성장세를 보였다. 필리핀은 총 4만1817명의 관광객이 방문하며 37% 늘었고, 태국과 베트남도 각각 15.1%, 17.5% 증가했다.미국과 캐나다를 포함한 미주 지역은 11만2483명으로 13.7%, 한류 붐이 일고 있는 러시아와 프랑스 등 유럽 지역은 10만4636명으로 21.5% 늘어났다.한편 지난달 해외여행을 떠난 우리 국민은 242만7634명으로 전년 대비 3.7% 감소했다.  

KeyboardInterrupt: 